In [0]:
dbutils.widgets.removeAll()

In [0]:
import json
from glob import glob

paths = glob('../layer_*/*.json')
schema_table_map = {}
for path in paths:
    with open(path) as f:
        settings = json.load(f)
    dst = settings.get('dst_table_name')
    if dst:
        catalog, schema, table = dst.split('.')
        schema_key = f"{catalog}.{schema}"
        schema_table_map.setdefault(schema_key, []).append(table)

for tables in schema_table_map.values():
    tables.sort()

schemas = sorted(schema_table_map.keys())

if schemas:
    dbutils.widgets.combobox('schema', schemas[0], schemas)
else:
    dbutils.widgets.combobox('schema', '', [])


In [0]:
schema = dbutils.widgets.get('schema')
tables = schema_table_map.get(schema, [])

if tables:
    dbutils.widgets.combobox('table', tables[0], tables)
else:
    dbutils.widgets.combobox('table', '', [])


In [0]:
schema = dbutils.widgets.get('schema')
tables = schema_table_map.get(schema, [])
records = []
for table in tables:
    hist = spark.sql(f"DESCRIBE HISTORY {schema}.{table}")
    if hist.count() > 0:
        latest = hist.agg({'version': 'max'}).collect()[0][0]
        records.append((table, int(latest)))
    else:
        records.append((table, None))
df = spark.createDataFrame(records, ['table_name', 'latest_version'])
display(df)


In [0]:
schema = dbutils.widgets.get('schema')
table = dbutils.widgets.get('table')
if schema and table:
    full_name = f"{schema}.{table}"
    hist = spark.sql(f"DESCRIBE HISTORY {full_name}")
    versions = [str(r['version']) for r in hist.select('version').collect()]
    versions = sorted(versions, key=int, reverse=True)
else:
    versions = []

if versions:
    dbutils.widgets.combobox('version', versions[0], versions)
else:
    dbutils.widgets.combobox('version', '', [])


In [0]:
schema = dbutils.widgets.get('schema')
table = dbutils.widgets.get('table')
version = dbutils.widgets.get('version')

In [0]:
full_table_name = f"{schema}.{table}"
print(f'Restoring {full_table_name} to version {version}')
if schema and table and version:
    spark.sql(f"RESTORE TABLE {full_table_name} TO VERSION AS OF {version}")
